# Documentacion

Esta es la documentacion cutre del programa. Espero que me sirva en
algun momento, pero dudo que sea vista por mis revisores, la
verdad...

## Indice

- [Input de usuario](#Input-de-usuario)
- [Definicion de elementos](#Definicion-de-elementos)
- [Cargas distribuidas](#Cargas-distribuidas)
- [Cargas nodales](#Cargas-nodales)
- [Analisis estatico](#Analisis-estatico)
- [Combinaciones de carga](#Combinaciones-de-carga)
- [Caso mas desfavorable](#Caso-mas-desfavorable)
- [Flujo de la aplicacion](#Flujo-de-la-aplicacion)

In [1]:
from numpy import array, zeros, ones, round as mround, vstack, reshape
from numpy.linalg import inv, solve
from math import pi as π, sin, cos
def line(): print('---- -- - -- ----')

## Input de usuario

- [volver](#Indice)

El usuario final es el encargado de llenar un documento de texto el
cual cuenta con la informacion principal para definir y generar una
estructura de marcos de hormigon armado. La estructura en cuestion,
para esta version del programa, se espera que conserve una simetria a
lo largo de sus ejes de manera independiente, en otras palabras, que
para ambas direcciones de analisis, todas las vigas deben ser iguales
entre si y lo mismo aplica para las columnas.

El input esperado por el usuario contempla la siguiente forma:

```
numero de pisos          : 1
numero habitaciones      :1
numero de marcos         :1
altura de piso           :300
distancia entre columnas :200
distancia entre marcos   :0

modulo de poisson 𝜈      :0.25
resistencia del hormigon :19.5
densidad del hormigon    : 2500
carga viva               :0

zona sismica             :
tipo de suelo            :
categoria de edificacion :

resistencia del acero    :
tama;o max de columnas   :
altura max de viga       :
```

La recuperacion de los datos del archivo ingresado por el usuario se
presenta en el siguiente bloque:

In [2]:
# recuperacion de datos

with open('../IO/input.txt', encoding='utf-8') as in_text:
    in_temp = in_text.readlines()

default = {
    'numero de pisos': '1',
    'numero habitaciones': '1',
    'numero de marcos': '1',
    'altura de piso': '300',
    'distancia entre columnas': '200',
    'distancia entre marcos': '0',
    'modulo de poisson 𝜈': '0.25',
    'resistencia del hormigon': '19.5',
    'densidad del hormigon': '2500',
    'carga viva': '0',
    'zona sismica': 'def',
    'tipo de suelo': 'def',
    'categoria de edificacion': 'def',
    'resistencia del acero': 'def',
    'tama;o max de columnas': 'def',
    'altura max de viga': 'def',
}

user_op = []
user_in = []

for i in in_temp:
    if i.startswith('\n') == False:
        user_op.append(i.partition(':')[0].strip())
        user_in.append(i.partition(':')[2].strip())
print(user_in)
del in_temp
if user_op != list(default.keys()): raise Exception('wena')

for i in range(len(user_in)):
    if user_in[i] == '':
        user_in[i] = list(default.values())[i]
    elif user_in[i].isalpha() == False:
        user_in[i] = float(user_in[i])
for i in range(3): user_in[i] = int(user_in[i])
print(user_in)

['1', '1', '1', '300', '200', '0', '0.25', '19.5', '2500', '0', '', '', '', '', '', '']
[1, 1, 1, 300.0, 200.0, 0.0, 0.25, 19.5, 2500.0, 0.0, 'def', 'def', 'def', 'def', 'def', 'def']


In [3]:
# Input desde archivo txt
zona = '3'
categoria = 'II'
suelo = 'E'
R = 5.5
h_pisos = 2.4 # [m] pero despues sera en cm y despues en varias unidades
n_pisos = 3
n_vanos = 3
h_losas = 20 # [cm]
VIVO = 20000 # [kg]

## Definicion de elementos

- [volver](#Indice)

En esta seccion, primero se define una clase con todos los atributos
de un elemento 'barra' para luego inicializar tantos elementos como
sean necesarios y se almacenan en una lista.

In [4]:
# definicion de elementos

class barra():
    # agregar rigidos
    def __init__(bar, nodos, i, j, x0, y0, x1, y1, b, h, ϰ, 𝜈, 𝛾, fc):
        """Define los parametros del perfil y material de un elemento 'barra'."""
        
        #### ## input ## ####
        bar.i = i # nodo A
        bar.j = j # nodo B
        bar.x0 = x0 # [cm]
        bar.y0 = y0 # [cm]
        bar.x1 = x1 # [cm]
        bar.y1 = y1 # [cm]
        bar.b = b # [cm]
        bar.h = h # [cm]
        bar.ϰ = ϰ # factor de forma de la seccion
        bar.𝜈 = 𝜈 # modulo de poisson
        bar.𝛾 = 𝛾 # [kg/m3] densidad
        bar.fc = fc # [MPa]
        
        if x0 != x1:
            bar.name = 'VIGA'
        else:
            bar.name = 'COL'

        #### ## internals ## ####
        bar.GdL(i, j, nodos) # grados de libertad

        bar.L = ((x1 - x0)**2 + (y1 - y0)**2)**0.5 # [cm]
        bar.s = (y1 - y0)/bar.L # seno
        bar.c = (x1 - x0)/bar.L # coseno
        bar.A = b*h # [cm2]
        bar.I = b*h**3/12 # [cm4]
        bar.E = 𝛾**1.5*0.043*fc**0.5/9.80665/0.01 # [kg/mm2]/0.01 = [kg/cm2]
        bar.G = bar.E/(2*(1+𝜈)) # modulo de cizalle
        bar.β = 6*bar.E*bar.I*ϰ/(bar.G*bar.A*bar.L**2)

        bar.rigidez_local(bar.L, bar.A, bar.I, bar.E, bar.β)
        bar.compatibilidad_geometrica(bar.s, bar.c, bar.L)

        bar.K = bar.a.T@bar.k@bar.a # rigidez global
        
    def GdL(bar, a, b, nodosEstruc, tipo=''):
        """Prepara la matriz de transformacion de cada barra."""
        bar.T = zeros([6, 3*nodosEstruc])
        if a > nodosEstruc or b > nodosEstruc:
            print('index error')
            return
        if a < 0 or b < 0:
            print('index error')
            return
        if a != 0:
            bar.T[0, 3*a - 3] = 1
            bar.T[1, 3*a - 2] = 1
            bar.T[2, 3*a - 1] = 1
        if b != 0:
            bar.T[3, 3*b - 3] = 1
            bar.T[4, 3*b - 2] = 1
            bar.T[5, 3*b - 1] = 1

    def compatibilidad_geometrica(bar, s, c, L):
        bar.a = array([
            [-s/L, c/L, 1,  s/L, -c/L, 0],
            [-s/L, c/L, 0,  s/L, -c/L, 1],
            [ c,   s,   0, -c,   -s,   0]
        ])

    def rigidez_local(bar, L, A, I, E, β):
        k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
        k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
        k21 = k12
        k22 = k11
        k33 = A*E/L
        bar.k = array([
            [k11, k12, 0  ],
            [k21, k22, 0  ],
            [0,   0,   k33]
        ])

    def addDistF(bar, a, b, fa, fb, coord='Y'):
        """Ingresa una fuerza distribuida desde 'a' hasta 'b'."""
        pass

    def sigma(bar, r_global): # sin uso - borrar
        # σ = k@ε
        # ε = a@r_i
        # r_i = T@r
        # σ = k@a@T@r
        bar.σ = bar.k@bar.a@bar.T@r_global # esfuerzos

    r = 0 # desplazamientos
    ε = 0 # deformaciones
    σ = 0 # esfuerzos [Ma, Mb, F]
    v = 0 # esfuerzo de corte
    d_load = [] # [kg/cm] - cargas distribuidas
    
    # despues agregar tramos rigidos

In [17]:
NODOS = 8 # cantidad de uniones de la estructura
b_VIG = 30
h_VIG = 40
b_COL = 30
h_COL = 30
ϰ = 0
𝜈 = 0.25
𝛾 = 2500
fc = 19.5

def miembros():
    """Genera todos los elementos 'barra' de la estructura."""
    barras = []
    m1 = barra(NODOS, 0, 1, 0, 0, 0, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m2 = barra(NODOS, 0, 2, 150, 0, 150, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m3 = barra(NODOS, 0, 3, 300, 0, 300, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m4 = barra(NODOS, 0, 4, 450, 0, 450, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m5 = barra(NODOS, 1, 5, 0, 120, 0, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m6 = barra(NODOS, 2, 6, 150, 120, 150, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m7 = barra(NODOS, 3, 7, 300, 120, 300, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m8 = barra(NODOS, 4, 8, 450, 120, 450, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
    m9  = barra(NODOS, 1, 2, 0, 120, 150, 120, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    m10 = barra(NODOS, 2, 3, 150, 120, 300, 120, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    m11 = barra(NODOS, 3, 4, 300, 120, 450, 120, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    m12 = barra(NODOS, 5, 6, 0, 240, 150, 240, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    m13 = barra(NODOS, 6, 7, 150, 240, 300, 240, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    m14 = barra(NODOS, 7, 8, 300, 240, 450, 240, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)

    for i in range(1, 15):
        exec('barras.append(m' + str(i) + ')')
    return barras

miembro = [None] + miembros()

K_s = zeros((3*NODOS, 3*NODOS)) # rigidez del sistema
for m in miembro:
    if m == None: continue
    K_s += m.T.T@m.K@m.T

p(miembro[3].a)
#dir()

[[-0.00833333  0.          1.          0.00833333 -0.          0.        ]
 [-0.00833333  0.          0.          0.00833333 -0.          1.        ]
 [ 0.          1.          0.         -0.         -1.          0.        ]]


## Cargas distribuidas

- [volver](#Indice)

Para cada elemento, se calculan las cargas distribuidas muertas y
vivas en funcion de la geometria del perimetro que forman asumiendo
que en cada elemento descarga una losa.

para el input de las cargas muertas es necesario conocer:

- distancia entre marcos

objetivo

- vector de cargas muertas

vector cargas muertas:

- peso propio marco
    - peso propio por elemento
- peso losas



In [6]:
# cargas vivas = input

In [7]:
# cargas muertas = peso propio

## Cargas nodales

- [volver](#Indice)

A partir de un analisis de una viga doblemente empotrada, se calculan
las reacciones de esta para cada caso, considerando la continuidad de
una losa que descargue sobre el marco. Para esto se consideran las
formulas de empotramiento perfecto de cargas lineales y continuas. De
este modo, se obtiene la carga trapezoidal resultante de la
superposicion de hasta tres casos de cargas por viga.

Formula para una carga constante:

In [8]:
def q_rec(L, q, a, b, d):
    """Calculo de las reacciones para una carga constante."""
    Ra = q*d/L**3*((2*a + L)*b**2 + (a - b)/4*d**2)
    Rb = q*d/L**3*((2*b + L)*a**2 - (a - b)/4*d**2)
    Ma = q*d/L**2*(a*b**2 + d**2/12*(L - 3*b))
    Mb = -q*d/L**2*(a**2*b + d**2/12*(L - 3*a))

    print(round(Ra, 1), round(Rb, 1), round(Ma, 1), round(Mb, 1))
    return Ra, Rb, Ma, Mb

l = 2
q = 1000
d = 1.2
a = 0.5 + d/2
b = l - a

q_rec(l, q, a, b, d);

521.1 678.9 242.1 -279.9


Formula para una carga lineal:

In [9]:
def q_tri(L, q, a):
    """Calculo de las reacciones para una carga lineal"""
    Ra = q*a/20*(10 - a**2/L**2*(15 - 8*a/L))
    Rb = q*a**3/(20*L**2)*(15-8*a/L)
    Ma = q*a**2/30*(10 - a/L*(15 - 6*a/L))
    Mb = -q*a**3/(20*L)*(5 - 4*a/L)
    print(round(Ra, 1), round(Rb, 1), round(Ma, 1), round(Mb, 1))
    return Ra, Rb, Ma, Mb

l = 2
q = 1000
a = 0.5
b = l - a

q_tri(l, q, a);

229.7 20.3 55.2 -12.5


La superposicion de las cargas lineales y constantes se lleva a cabo
en el siguiente bloque (pero no recuerdo como funciona xd):

In [10]:
# superposicion de casos de carga de una viga cualquiera

def nomeacuerdodeesto():
    r = [[],[],[]]
    joints = []

    L = 2
    a = 0.5
    b = 0.3
    d = 1.2
    q = 1000

    *r[0], = q_tri(L, q, a)
    *r[1], = q_rec(L, q, a + d/2, b + d/2, d)
    *r[2], = q_tri(L, q, b)

    r[2][0], r[2][1] = r[2][1], r[2][0]
    r[2][2], r[2][3] = -r[2][3], -r[2][2]

    for i in range(4):
        joints.append(0)
        for j in range(3):
            joints[i] += r[j][i]

    line()

    print('Va:', round(joints[0], 1))
    print('Vb:', round(joints[1], 1))
    print('Ma:', round(joints[2], 1))
    print('Mb:', round(joints[3], 1))

## Analisis estatico

- [volver](#Indice)

Se genera un vector de fuerzas solicitantes, E, a partir del metodo
estatico.

In [11]:
# cargas sismicas - metodo estatico
def metodo_estatico(zona, categoria, suelo, R, h_pisos, n_pisos, n_vanos):
    """Calcula un vector con las fuerzas horizontales producto del sismo."""
    TABLA_I = { # Tabla 6.1
        # "CategoriaDeEdificio": I,
        "I": 0.6,
        "II": 1.0,
        "III": 1.2,
        "IV": 1.2,
    }

    TABLA_A0 = { # Tabla 6.2
        # "ZonaSismica": A0/g,
        "1": 0.20,
        "2": 0.30,
        "3": 0.40,
    }

    TABLA_PARAMETROS_SUELO = { # Tabla 6.3
        # "TipoDeSuelo": (S, T0, Tp [T'], n, p),
        "A": (0.90, 0.15, 0.20, 1.00, 2.0),
        "B": (1.00, 0.30, 0.35, 1.33, 1.5),
        "C": (1.05, 0.40, 0.45, 1.40, 1.6),
        "D": (1.20, 0.75, 0.85, 1.80, 1.0),
        "E": (1.30, 1.20, 1.35, 1.80, 1.0),
        "F": (None, None, None, None, None),
    }

    TABLA_C_MAX = { # Tabla 6.4
        # "R": CoeficienteC_max,
        "2": 0.90,
        "3": 0.60,
        "4": 0.55,
        "5.5": 0.40,
        "6": 0.35,
        "7": 0.35,
    }

    Tn = round(0.012*h_pisos*n_pisos, 4)

    A0 = TABLA_A0[zona] # aceleracion efectiva
    I = TABLA_I[categoria]
    S, T0, Tp, n, p = TABLA_PARAMETROS_SUELO[suelo]

    # coeficiente sismico
    Cmax = round(TABLA_C_MAX[str(R)]*S*A0, 8)
    Cmin = A0*S/6
    C = 2.75*S*A0/R*(Tp/Tn)**n

    if C < Cmin: C = Cmin
    elif C > Cmax: C = Cmax

    VIVO = 20000 # [kg]
    MUERTO = 40000 # [kg]
    Pk = ( # peso propio por nivel
        0, # nivel 0 (suelo)
        0.25*VIVO + MUERTO,
        0.25*VIVO + MUERTO,
        0.25*VIVO + MUERTO/2,
    )

    P = 0 # [kg] - peso sismico
    for i in Pk:
        P += i

    Q0 = C*I*P # [kg] corte basal

    print(A0, I, S, T0, Tp, n, p, Cmax, round(Cmin, 4), round(C, 4), Q0, Pk, P) # borrame

    H = round(h_pisos*n_pisos, 4)
    Z = [0] + [round(i*h_pisos, 4) for i in range(1, n_pisos + 1)]
    A = [0]
    for k, _ in enumerate(Z):
        if k == 0: continue
        A.append(
            (1 - Z[k - 1]/H)**0.5 - (1 - Z[k]/H)**0.5
        )
        print(k, Z[k], round(A[k], 3)) # borrame

    P = Pk
    del Pk

    F = []
    AjPj = sum([i*k for i, k in zip(A, P)])
    for k, _ in enumerate(P): F.append(A[k]*P[k]/AjPj*Q0)

    print(AjPj) # borrame
    print((F)) # borrame

    fx = []
    for k, _ in enumerate(F):
        if k == 0: continue
        for i in range(n_vanos + 1):
            fx.append(F[k]/(n_vanos + 1))
            fx.append(0)
            fx.append(0)
    return array(fx).T

E = metodo_estatico(zona, categoria, suelo, R, h_pisos, n_pisos, n_vanos)

0.4 1.0 1.3 1.2 1.35 1.8 1.0 0.208 0.0867 0.208 23920.0 (0, 45000.0, 45000.0, 25000.0) 115000.0
1 2.4 0.184
2 4.8 0.239
3 7.2 0.577
33452.99461620748
[0.0, 5904.496220906295, 7694.889288930095, 10320.614490163613]


## Combinaciones de carga

- [volver](#Indice)

Se genera un vector de fuerzas, _S_, para cada combinacion de cargas.

In [12]:
# vector de fuerzas segun casos de carga
    # 1.4*D
    # 1.2*D + 1.6*L
    # 1.2*D + 1.4*E + L
    # 1.2*D - 1.4*E + L
    # 0.9*D + 1.4*E
    # 0.9*D - 1.4*E

S = array([500, -1000, 0,
           500, -1000, 0,
           500, -1000, 0,
           500, -1000, 0,
           500, -1000, 0,
           500, -1000, 0,
           500, -1000, 0,
           500, -1000, 0,
    ]).T

## Caso mas desfavorable

- [volver](#Indice)

Se busca la combinacion de carga que controla el diseño, luego se
busca en la estructura la viga y la columna mas desfavorable. En el
caso de la viga, se calcula su momento al medio del elemento.

In [13]:
r = inv(K_s)@S # desplazamientos

for m in miembro: # finishing bars attributes
    if m == None: continue
    m.r = m.T@r
    m.ε = m.a@m.r
    m.σ = m.k@m.ε
    m.v = (m.σ[0] + m.σ[1])/m.L

print('Ma:', round(miembro[1].σ[0]/100, 0), round(miembro[7].σ[0]/100, 0), round(miembro[10].σ[0]/100, 0), '[kg-m]')
print('Mb:', round(miembro[1].σ[1]/100, 0), round(miembro[7].σ[1]/100, 0), round(miembro[10].σ[1]/100, 0), '[kg-m]')
print('F:', round(miembro[1].σ[2], 0), round(miembro[7].σ[2], 0), round(miembro[10].σ[2], 0), '[kg]')
print('V:', round(miembro[1].v, 0), round(miembro[7].v, 0), round(miembro[10].v, 0), '[kg]')

Ma: 620.0 359.0 -463.0 [kg-m]
Mb: 469.0 402.0 -463.0 [kg-m]
F: 930.0 995.0 -0.0 [kg]
V: 908.0 634.0 -617.0 [kg]


In [14]:
# viga y columna mas desfavorable

worst_v = (0, 0, 0)
worst_c = (0, 0, 0)

for i, m in enumerate(miembro):
    if m == None: continue
    if m.name != 'VIGA': continue
    if m.σ[0] + m.σ[1] > worst_c[0] + worst_c[1]: worst_c = (m.σ[0], m.σ[1], i)

for i, m in enumerate(miembro):
    if m == None: continue
    if m.name != 'COL': continue
    if m.σ[0] + m.σ[1] > worst_v[0] + worst_v[1]: worst_v = (m.σ[0], m.σ[1], i)

In [15]:
# momento max al centro

## Flujo de la aplicacion

- [volver](#Indice)

A continuacion, se resume el cuerpo de la aplicacion, donde se
escribe de forma declarativa el flujo del programa.

In [16]:
def pAPP():
    leerInput()
    armarEstructura()
    optimus = False
    while not optimus:
        cargarCombos()
        anal()
        optijandro()
        if jandro_is_true:
            optimus = True
        else:
            nuevoPerfil()
    reporte()
    return 0

#ned